In [100]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time

# Classes

In [101]:
class Individual:
  """
  A class that represents an individual solution.

  Attributes:
    chromosome (1D list): The chromosome of the individual in [0,1] interval.
    weights (2D list): The weights of the individual in [-1,1] interval.
    fitness (float): The fitness of the individual.
  """

  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, fitness:float = math.inf):
    self._chromosome = chromosome
    self._weights = weights
    
    if fitness < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = fitness
  
  def __str__(self):
    return f"""
    Individual:
        Feature-selection: {self._chromosome}
        Weights: {self._weights}
        Fitness: {self._fitness}
    """
  @property
  def chromosome(self):
    return self._chromosome
  
  @property
  def weights(self):
    return self._weights

  @property
  def fitness(self):
    return self._fitness

  @chromosome.setter
  def chromosome(self, chromosome):
    self._chromosome = chromosome

  @weights.setter
  def weights(self, weights):
    self._weights = weights

  @fitness.setter
  def fitness(self,f):
    self._fitness = f

In [102]:
class Particle(Individual):
  """
  A class that represents an individual solution in Swarm-based problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    best_fitness (float): The best fitness of the individual.
    best_weights_position (numpy.ndarray): The best weight position of the individual.
    best_chromosome_position (numpy.ndarray): The best chromosome position of the individual.
    weights_velocity (numpy.ndarray): The velocity of the weights of the individual.
    chromosome_velocity (numpy.ndarray): The velocity of the chromosome of the individual.  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, fitness:float = math.inf):
    Individual.__init__(self, chromosome, weights, fitness)
    self._best_fitness = fitness # At the beginning, the best fitness is the input fitness
    self._best_weights_position = weights # By default, the best weight position is the starting weight position.
    self._best_chromosome_position = chromosome # By default, the best chromosome position is the starting chromosome position.
    self._weights_velocity = np.zeros(weights.shape) # All swarm's weights velocities are set to 0 at the start
    self._chromosome_velocity = np.zeros(chromosome.shape) # All swarm's chromosome velocities are set to 0 at the start

  def __str__(self):
    return f"""
    Particle:
        Feature-selection: {self.chromosome}
        Weights: {self.weights}
        Fitness: {self.fitness}
        Actual weights velocity: {self._weights_velocity}
        Actual feature-selection velocity: {self._chromosome_velocity}
        Best fitness: {self._best_fitness}
        Best weights: {self._best_weights_position}
        Best feature-selection: {self._best_chromosome_position}
    """

  @property
  def best_fitness(self):
    return self._best_fitness

  @property
  def best_weights_position(self):
    return self._best_weights_position
  
  @property
  def best_chromosome_position(self):
    return self._best_chromosome_position
  
  @property
  def weights_velocity(self):
    return self._weights_velocity

  @property
  def chromosome_velocity(self):
    return self._chromosome_velocity

  @best_fitness.setter
  def best_fitness(self,f):
    self._best_fitness = f

  @best_weights_position.setter
  def best_weights_position(self,w):
    self._best_weights_position = w

  @best_chromosome_position.setter
  def best_chromosome_position(self,c):
    self._best_chromosome_position = c

  @weights_velocity.setter
  def weights_velocity(self,w):
    self._weights_velocity = w

  @chromosome_velocity.setter
  def chromosome_velocity(self,c):
    self._chromosome_velocity = c

In [103]:
class Population:
  """
  A class that represents a population of individual solutions.

  Attributes:
  -----------
    size (int): The size of the population.
    gene_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
  
  Methods:
  -----------
    insert_best_gene(Individual): Inserts the best individual of the population into the gene_list.
    add_gene_to_list(Individual): Adds an individual to the gene_list.
  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    self._size = size
    self._best_gene = None

    if is_empty == True:
      self._genes_list = np.full(shape= size, fill_value= None)
    else:
      if is_swarm == True:
        self._genes_list = np.full(shape= size, fill_value= create_particle(K,D))
      else:
        self._genes_list = np.full(shape= size, fill_value= create_individual(K,D))
  
  def __str__(self):
    return f"""
    Population:
        Size: {self._size}
        Gene list: {self._genes_list}
        Best gene: {self._best_gene}
    """
  @property
  def size(self):
    return self._size
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def best_gene(self):
    return self._best_gene

  @best_gene.setter
  def best_gene(self,gen):  
    self._best_gene = gen

  # Methods
  def add_gene_to_list(self,gen, index:int):
    self._genes_list = np.insert(self._genes_list, index, gen)

  def insert_best_gene(self,gen):
    self.best_gene = gen
    self._genes_list = np.insert(self._genes_list, 0, gen) # Insert the best gene at the beginning of the list

In [135]:
empty_array = np.empty(shape=[10], dtype=object)

print(f"Empty array: {empty_array} with shape: {empty_array.shape}")

empty_array = np.insert(empty_array, 1, 1)
empty_array = np.insert(empty_array, 1, 2)

print(f"Empty array: {empty_array} with shape: {empty_array.shape}")



Empty array: [None None None None None None None None None None] with shape: (10,)
Empty array: [None 2 1 None None None None None None None None None] with shape: (12,)


In [104]:
class Swarm(Population):
  """
  A class that represents a population of solutions in Swarm-based problems.

  Attributes:
  -----------
    (same as Population), and additional attributes:

    global_best_fitness (float): The best fitness of the population.
    global_best_weights (numpy.ndarray): The best weight position of the population.
    global_best_chromosome (numpy.ndarray): The best chromosome position of the population.    
  
  Methods:
  -----------

  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    Population.__init__(self, size, K, D, is_empty, is_swarm=True)
    self._global_best_fitness = math.inf
    self._global_best_weights = None
    self._global_best_chromosome = None

  def __str__(self):
    return f"""
    Swarm:
        Size: {self._size}
        Particles list: {self.genes_list}
        Best particle: {self.best_gene}
        Global best fitness: {self._global_best_fitness}
        Global best weights: {self._global_best_weights}
        Global best feature-selection: {self._global_best_chromosome}
    """
  @property
  def global_best_fitness(self):
    return self._global_best_fitness

  @property
  def global_best_weights(self):
    return self._global_best_weights

  @property
  def global_best_chromosome(self):
    return self._global_best_chromosome

  @global_best_fitness.setter
  def global_best_fitness(self,f):
    self._global_best_fitness = f
  
  @global_best_weights.setter
  def global_best_weights(self,w):
    self._global_best_weights = w
  
  @global_best_chromosome.setter
  def global_best_chromosome(self,c):
    self._global_best_chromosome = c

In [105]:
class Reef:
  """
  A class that represents a population of solutions in Evolutionary-based problems.

  Attributes:
  ----------- 
    size (int): The size of the population.
    free_occupied_rate (float): The rate of occupied individuals in the reef.
    genes_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
    
  Methods:
  -----------
  """

  def __init__(self, size:int, rate:float, K:int, D:int):
    self._size = size
    self._free_occupied_rate = rate
    self._best_coral = None
    self._corals_list = np.full(shape = [size], fill_value = None) # Empty reef

    occupiedHoles = int(size * rate) # Partially occupy the reef
    self._corals_list[:occupiedHoles] = [create_individual(K,D) for _ in range(occupiedHoles)] # Fill the reef with individuals

  def __str__(self):
    return f"""
    Reef:
        Size: {self._size}
        Free-occupied rate: {self._free_occupied_rate}
        Corals list: {self._corals_list}
        Best coral: {self._best_coral}
    """
  
  @property
  def size(self):
    return self._size

  @property
  def free_occupied_rate(self):
    return self._free_occupied_rate
  
  @property
  def corals_list(self):
    return self._corals_list

  @property
  def best_coral(self):
    return self._best_coral
  
  @corals_list.setter
  def corals_list(self,corals_list):
    self._corals_list = corals_list

  @best_coral.setter
  def best_coral(self,coral):
    self._best_coral = coral
  
  # Methods
  def insert_new_larvae_in_hole(self, larvae: Individual, hole_index:int):
    self.corals_list[hole_index] = larvae
    self.fitness_list[hole_index] = larvae.fitness

  def remove_coral_from_hole(self, hole_index:int):
    self.corals_list[hole_index] = None
    self.fitness_list[hole_index] = math.inf
  
  def sort_by_fitness(self):
    # Sort coral_list by fitness in descending order
    self.corals_list = self.corals_list[np.argsort(self.fitness_list)[::-1]]


# METHODS

In [106]:
def sumMatrixes(mat1,mat2):
  return np.add(mat1,np.asarray(mat2))

def subtractMatrixes(mat1,mat2):
  return np.subtract(mat1,np.asarray(mat2))

def dotMultiply(mat1,mat2):
  return np.multiply(mat1,np.asarray(mat2))

def multiplyMatrixes(mat1,mat2):
  return np.matmul(mat1,np.asarray(mat2))

def dotDivide(mat1,mat2):
  return np.divide(mat1,np.asarray(mat2))

In [107]:
def random_chromosome_generator(K:int):
  return np.random.randint(low= 0, high= 1 + 1, size = K) # 0 or 1

def random_weights_generator(K:int, D:int):
  return np.random.uniform(low= -1, high= 1, size=(K,D)) # -1 to 1

In [108]:
def create_individual(K:int = 0, D:int = 0):
  return Individual(random_chromosome_generator(K), random_weights_generator(K,D))

def create_particle(K:int = 0, D:int = 0):
  return Particle(random_chromosome_generator(K), random_weights_generator(K,D))  

def create_population(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
  return Population(size,K,D, is_empty) 

def create_swarm(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
  return Swarm(size,K,D, is_empty) 

def create_reef(size:int, rate:float, K:int = 0, D:int = 0):
  return Reef(size, rate, K, D) 

In [109]:
def InternalReproduction(gen: Individual):
  # Mutate chromosome and weights
  mutate_point = np.random.randint(0, len(gen.chromosome)) # Choose a random point to mutate
  gen.chromosome[mutate_point] = 1 - gen.chromosome[mutate_point] # Mutate the point
  
  # Mutate weights
  total_rows = gen.weights.shape[0] 
  total_columns = gen.weights.shape[1] 
  mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate
  
  gen.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

In [110]:
def ExternalReproduction(father:Individual, mother:Individual):
  crossover_point = np.random.randint(0, len(father.chromosome))
  crossover_column = random.randint(0, len(father.weights[0]))
  
  # Offsprings are created by combining father and mother chromosomes and weights
  offspring1 = Individual( chromosome = np.concatenate((father.chromosome[:crossover_point], mother.chromosome[crossover_point:])), 
                           weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1) ) 
  
  offspring2 = Individual( chromosome = np.concatenate((mother.chromosome[:crossover_point], father.chromosome[crossover_point:])), 
                           weights = np.concatenate((mother.weights[:,:crossover_column], father.weights[:,crossover_column:]), axis=1) ) 
                       
  return [offspring1,offspring2] 

In [111]:
def RouletteWheelSelection(population:Population): 
  # Create an empty population for the chosen individuals of the Roulette Wheel
  roulettePopulation = create_population(population.size, is_empty=True)
  roulettePopulation.insert_best_gene(population.best_gene) # Make sure we don't lose our best gene in the roulette 

  print("... Roulette Wheel Selection ...")
  
  # Total population fitness (S)
  S = sum([individual.fitness for individual in population.genes_list])

  # Population chromosomes' relative probabilities
  rel_prob = [individual.fitness/S for individual in population.genes_list]

  for _ in range(population.size - 1): # -1 because we already inserted the best gene
    r = np.random.uniform() 

    # Find the first index for which q_i < r
    for index,individual in enumerate(population.genes_list): 
      r -= rel_prob[index] 
      if r < 0:
        roulettePopulation.add_gene_to_list(individual, index) # Add the individual to the roulette population
        break
    # end choose rouletted individual loop
  # end roulette population loop

  return roulettePopulation  

In [112]:
''' This functions reproduces a population.
    It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
    Returns the input population, updated with the crossover and mutated childs '''

def ReproducePopulation(population: Population):
  childs = [] # list to store future offsprings
  crossover_probability = np.random.uniform()
  mutation_probability = np.random.uniform()
  gene_list_size = len(population.genes_list)

  print("... Reproducing population ...")
  for index,individual in enumerate(population.genes_list):
    # Generate a random mother if the crossover probability is met
    if(np.random.random() >= crossover_probability):
      random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother

      while(random_mother_index == index): # If the mother is the same as the father, choose another one
        random_mother_index = np.random.randint(0,gene_list_size)
      #end while
      
      # Crossover parents
      mother = population.genes_list[random_mother_index]
      offsprings = ExternalReproduction(individual, mother)

      # Mutate offsprings if probability is over the mutation probability
      rand_value = np.random.random()
      if(rand_value >= mutation_probability): 
        InternalReproduction(offsprings[0])

      if(rand_value >= mutation_probability):
        InternalReproduction(offsprings[1])

      childs.append(offsprings[0])
      childs.append(offsprings[1])
  
  # Extend the population with the new childs
  population.genes_list.extend(childs)
  return population

In [113]:
def UpdateVelocitiesAndPositionsPSO(swarm: Swarm, w:float, c1:float, c2:float):
  for index, particle in enumerate(swarm.genes_list): # For each particle in the swarm

    # Velocities
    r1 = np.random.uniform()
    r2 = np.random.uniform()

    # Chromosome loop
    for k in range(len(particle.chromosome)):
      # Update chromosome velocity 
      particle.chromosome_velocity[k] = ( 
                                          (w * particle.chromosome_velocity[k]) +
                                          (c1*r1*(particle.best_chromosome_position[k] - particle.chromosome[k])) +
                                          (c2*r2*(swarm.global_best_chromosome[k] - particle.chromosome[k]))
                                        )

      # Prevent velociy from going out of bounds
      if(particle.chromosome_velocity[k] > 6):
        particle.chromosome_velocity[k] = 6
      elif(particle.chromosome_velocity[k] < -6):
        particle.chromosome_velocity[k] = -6

      velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*particle.chromosome_velocity[k]) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1

      # Update to next position
      if(np.random.uniform() < velocityProbability):
        particle.chromosome[k] = 1
      else:
        particle.chromosome[k] = 0
    # end chromosome loop

    # Weight loop
    for i in range(particle.weights.shape[0]):
      for j in range(particle.weights.shape[1]):

        particle.weights_velocity[i,j] = ( 
                                          (w * particle.weights_velocity[i,j]) +
                                          (c1*r1*(particle.best_weights_position[i,j] - particle.weights[i,j])) +
                                          (c2*r2*(swarm.global_best_weights[i,j] - particle.weights[i,j]))
                                        )
        # Update to next position
        particle.weights[i,j] += particle.weights_velocity[i,j]
    # end weight loop
  # end particle loop
  return swarm

In [114]:
def split_reef_candidates(candidates:list, fraction:float):
    number_of_candidates = len(candidates)
    split_point = int(number_of_candidates * fraction)

    # Split candidates in two lists
    broadcast_candidates = candidates[:split_point]
    brooding_candidates = candidates[split_point:]
    
    # If broadcast candidates are odd, add one more candidate to the list and remove it from the brooding candidates
    if len(broadcast_candidates) % 2 != 0:
        broadcast_candidates.append(candidates[split_point])
        brooding_candidates.remove(candidates[split_point])

    return [broadcast_candidates, brooding_candidates]

# DATA PROCESSING

In [115]:
# SETTINGS CONSTANTS
CSV_NAME = "parkinsons.csv"
PORTATIL_DEV = False

if PORTATIL_DEV:
    CSV_PATH = "C:/Users/david/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
else:
    CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
    
# - - - - - - 
df = pd.read_csv(CSV_PATH + CSV_NAME, sep=" ", header=None)
end = df.shape[1]

# DATAFRAME CONSTANTS
X = df.iloc[:, :end-1]
Y = df.iloc[:, end-1] 

X = X.to_numpy() # IMPORTANT, otherwise we would be using DataFrames/Series instead of numpy arrays
Y = Y.to_numpy()
J = len(np.unique(Y)) # Number of classes
N,K = X.shape[0],X.shape[1]  # N = number of samples, K = number of features

In [116]:
# Scaling X (min-max normalization)
X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# First partition training/test
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # 20% test data

# Second partition validation
X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # 20% validation data

# ELM

In [117]:
# Computes the fitness for a given Individual using Extreme Learning Machine (ELM) model
def ComputeIndividualFitness(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # W
    individualWeights = dotMultiply(individual.weights, individual.chromosome[:, np.newaxis])

    # Feature-selection Bias given Gene's chromosome
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, individualWeights) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, individualWeights) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Output
    Y_predicted = multiplyMatrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [118]:
# Computes and udpates the fitnesses of a given Population 
def ComputePopulationFitness(population: Population, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  print("\n... Computing fitnesses ...")
  for individual in population.genes_list:
    
    fitness_i = ComputeIndividualFitness(individual, D, C, trainingX, trainingY, testX, testY)
    individual.fitness = fitness_i # Update fitness
    population.add_fitness_to_list(fitness_i)

    # Update best gene in Population
    if population.best_gene is None:
      population.best_gene = individual
    else:
      if individual.fitness < population.best_gene.fitness:
        population.best_gene = individual
  # end for

In [119]:
# Computes and udpates the fitnesses of a given Swarm
def ComputeSwarmFitness(swarm: Swarm, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  print("\n...Computing fitnesses...")
  for particle in swarm.genes_list:  

    fitness_i = ComputeIndividualFitness(particle, D, C, trainingX, trainingY, testX, testY) 
    particle.fitness = fitness_i
    swarm.add_fitness_to_list(fitness_i)

    # Update personal bests and global best
    if particle.fitness < particle.best_fitness:
      particle.best_fitness = particle.fitness
      
      if swarm.best_gene is None:
        swarm.best_gene = particle
      else:
        if particle.fitness < swarm.best_gene.fitness:
          swarm.best_gene = particle

  # Set Swarm's best chromosome and weights
  swarm.global_best_chromosome = swarm.best_gene.chromosome
  swarm.global_best_weights = swarm.best_gene.weights
  swarm.global_best_fitness = swarm.best_gene.fitness

  print("Global best fitness: ", swarm.global_best_fitness)

In [120]:
# Computes and udpates the fitnesses of a given Reef
def ComputeReefFitness(reef: Reef, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  print("\n...Computing fitnesses...")
  for index,coral in enumerate(reef.corals_list):  
    if coral is None:
      continue

    fitness_i = ComputeIndividualFitness(coral, D, C, trainingX, trainingY, testX, testY) 
    coral.fitness = fitness_i

    # Update best gene in Reef
    if reef.best_coral is None:
      reef.best_coral = coral
    else:
      if coral.fitness < reef.best_coral.fitness:
        reef.best_coral = coral

In [121]:
# Computes and udpates the fitnesses of a given Pool list
def ComputePoolFitness(pool: list, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    for larvae in pool:  
        fitness_i = ComputeIndividualFitness(larvae, D, C, trainingX, trainingY, testX, testY) 
        larvae.fitness = fitness_i
    # end loop   
    return pool

In [122]:
def TrainModelAndOutputResults(best_weigths:np.ndarray, best_chromosome:np.ndarray, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # Feature-selected weights
    weights = dotMultiply(best_weigths, best_chromosome[:, np.newaxis]) 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiplyMatrixes(trainingX, weights) + BiasTrainingMatrix
    activationTest = multiplyMatrixes(testX, weights) + BiasTestMatrix
    
    H_Training = dotDivide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dotDivide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiplyMatrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dotDivide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiplyMatrixes(inverse, np.transpose(H_Training))
    Beta = multiplyMatrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiplyMatrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

# BIO-INSPIRED ALGORITHMS

In [123]:
''' This functions apply the Evolutionary-based Genetic Algorithm to a given population '''
def ga(population: Population, max_generations:int, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    print("- - - - - - - - - - - - - - - - -")
    print("Generation: ", t)
    
    # Reproduction
    ReproducePopulation(population)

    # Compute Fitness
    ComputePopulationFitness(population, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Roulette Selection
    population = RouletteWheelSelection(population)

    # Continue iterating
    t += 1
  # end while 
  return population

In [124]:
''' This method apply the Swarm-based algorithm of PSO to a given swarm'''
def pso(swarm: Swarm, max_generations:int, w_max:float,  w_min:float, c1:float, c2:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    print("- - - - - - - - - - - - - - - - -")
    print("Generation: ", t)

    # Linearly decrease the inertia weight
    w = w_max - (w_max - w_min) * t / max_generations

    # Evaluate Particles' Fitness and update Pb_i and Gb
    ComputeSwarmFitness(swarm, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)
  
    # Update Velocity and Position of each particle
    UpdateVelocitiesAndPositionsPSO(swarm, w, c1, c2)

    # Increase step
    t += 1

    print(f"Global best fitness {swarm.best_gene.fitness} - at generation {t}")
    print(f"Global best gene fitness {swarm.best_gene.fitness} - at generation {t}")

  # end while
  return swarm

In [125]:
def cro(reef:Reef, max_generations: int, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  # Evaluate reef at the beginning
  ComputeReefFitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

  t = 0

  depredation_probability = 0.2 # predation probability

  while t < max_generations:
    print("- - - - - - - - - - - - - - - - -")
    print("Iteration:", t)

    # Update variables
    pool = [] # pool of candidates to select from
    f_broadcast = np.random.uniform() # broadcast fraction
    f_asexual = np.random.uniform() # asexual fraction
    f_depredation = f_asexual

    if (f_depredation + f_asexual > 1):
      f_depredation = f_asexual = 0.5


    print("... Starting reproduction phase ...")
    auxiliar_reef = reef.corals_list.copy() # Copy reef to avoid changing the original individuals

    # Get reproduction candidates
    reproduction_candidates = [c for c in auxiliar_reef if c is not None]
    [broadcast_set, brooding_set] = split_reef_candidates(reproduction_candidates, f_broadcast)

    # Broadcast    
    if len(broadcast_set) > 0: # If broadcast set is not empty, crossover
      while len(broadcast_set) > 0:
        # Select random father and a mother to crossover
        [father_candidate, mother_candidate] = np.random.choice(broadcast_set, 2, replace=False)
        offsprings = ExternalReproduction(father = father_candidate, 
                                          mother =  mother_candidate)

        # Add offspring to the pool
        pool.append(offsprings[0]) # Two parents reproduce only one coral larva
        
        # Remove fathers of the reproduction list
        broadcast_set.remove(father_candidate)
        broadcast_set.remove(mother_candidate)
    
    #  Brooding
    if len(brooding_set) > 0: # If broadcast set is not empty, mutate each individual
      for index, c in enumerate(brooding_set):
        InternalReproduction(c)
        # Add mutated offspring to the pool
        pool.append(c)

    # Larvae settings
    pool = ComputePoolFitness(pool, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)
    
    for larvae in pool:
      random_hole = np.random.randint(0, reef.size)
      
      if reef.corals_list[random_hole] is None:
        reef.insert_new_larvae_in_hole(larvae, random_hole)

      elif reef.corals_list[random_hole] is not None and larvae.fitness < reef.corals_list[random_hole].fitness:
        reef.insert_new_larvae_in_hole(larvae, random_hole)
    
    # Asexual reproduction
    reef.sort_by_fitness()

    for idx,c in enumerate(reef.corals_list):
      if c is not None:
        print(f"Coral {idx} after sort fitness - {c.fitness}")
      else:
        print(f"Coral {idx} after sort fitness - None")
        
    corals_in_reef = [c for c in reef.corals_list if c is not None]
    asexual_set = corals_in_reef[: int(f_asexual * len([coral for coral in reef.corals_list if coral is not None]))] 

    for idx,c in enumerate(asexual_set):
      print(f"Asexual {idx} fitness - {c.fitness}")
    print("-------------------------------")

    # Predation
    '''
    predation_number = int(f_depredation * len([coral for coral in reef.corals_list if coral is not None]))
    for i in range(predation_number):
      # Remove the last corals if the depression probability is met
      if np.random.uniform() < depredation_probability:
        reef.corals_list.remove(reef.corals_list[-1])
        reef.corals_list.append(None)

    # Fitness Evaluation

    '''
    # Increase step
    t += 1
  #end while
  return reef

# MAIN

In [126]:
# HYPERPARAMETERS FOR RUNNING SCRIPTS

# GENERAL
# K - declared already as the number of features
OPTIMAL_D = 10 
OPTIMAL_C = 1 


# GA
RUN_GA = True
size_population = 4
max_generations = 3
crossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population function
mutation_prob = 0.15

# PSO
RUN_PSO = True
swarm_size = 200 # total population
num_generations = 40
w_max = 1 # inertia weight coefficient (max)
w_min = 0.3 # inertia weight coefficient (min)
c1 = 2 # cognitive coefficient
c2 = 2 # social coefficient

# CRO
RUN_CRO = True    
reef_size = 10
rho0 = 0.8 # free/occupied rate
eta = 2 # larvae's attempts
coral_generations = 5
fraction_broadcast = np.random.uniform() # broadcast fraction
fraction_brooding = 1 - fraction_broadcast # brooding fraction
predation_percentage = 0.2 # predation probability


In [127]:
# GA
if RUN_GA == True:
    print("Starting Genetic Algorithm ...")
    print(f"Creating population with {size_population} individuals ... K = {K} - D = {OPTIMAL_D}")
    ga_start_execution_time = time.time()

    # Init population
    first_population = create_population(size_population, K, OPTIMAL_D)

    
    # Apply Algorithm and Evolve
    lastGeneration = ga(first_population, max_generations, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)


    # Train ELM with best W
    GA_CCR = TrainModelAndOutputResults(best_weigths = lastGeneration.best_gene.weights, 
                                        best_chromosome = lastGeneration.best_gene.chromosome, 
                                        D = OPTIMAL_D,
                                        C = OPTIMAL_C, 
                                        trainingX = X_train,
                                        trainingY = Y_train,
                                        testX = X_test,
                                        testY = Y_test)
                                        
    print("GA execution time: ", time.time() - ga_start_execution_time, "sec")
    print("Best last generation individual: ",lastGeneration.best_gene)
    print("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)
    print("- - - - - - - -")

Starting Genetic Algorithm ...
Creating population with 4 individuals ... K = 21 - D = 10
- - - - - - - - - - - - - - - - -
Generation:  0
... Reproducing population ...


AttributeError: 'numpy.ndarray' object has no attribute 'extend'

In [ ]:
# PSO
if RUN_PSO == True:
    print("Starting Particle Swarm Optimization algorithm...")
    print(f"Creating swarm with {swarm_size} individuals ... K = {K} - D = {OPTIMAL_D}")
    pso_start_execution_time = time.time()
    
    # Init population
    first_swarm = create_swarm(swarm_size, K, OPTIMAL_D)

    # Apply Algorithm and Iterate
    final_swarm = pso(first_swarm, num_generations, w_max, w_min, c1, c2, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)

    # Train ELM with best W
    PSO_CCR = TrainModelAndOutputResults(best_weigths = final_swarm.global_best_weights, 
                                         best_chromosome = final_swarm.global_best_chromosome, 
                                         D = OPTIMAL_D,
                                         C = OPTIMAL_C, 
                                         trainingX = X_train,
                                         trainingY = Y_train,
                                         testX = X_test,
                                         testY = Y_test)

    # Output results
    #print(f"Best solution found: {final_swarm.best_gene}")
    #print(f"Maximum weight value found: {final_swarm.best_gene.weights.max()} - Minimum weight value found: {final_swarm.best_gene.weights.min()}")
    #print(f"Maximum velocities value found: {final_swarm.best_gene.weights_velocity.max()} - Minimum velocities value found: {final_swarm.best_gene.weights_velocity.min()}")
    print("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)
    print("PSO execution time: ", time.time() - pso_start_execution_time, "sec")

In [ ]:
# CRO
if RUN_CRO == True:
    print("Starting Coral Reef Optimization algorithm...")
    print(f"Creating reef with {reef_size} holes and {rho0*reef_size} occupied spaces...")
    cro_start_execution_time = time.time()

    # Init population
    reefFirstGeneration = create_reef(reef_size, rho0, K, OPTIMAL_D)

    # Apply CRO algorithm and evolve
    last_reef = cro(reefFirstGeneration, coral_generations, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)

    print("CRO Completed")
    print("CRO execution time: ", time.time() - cro_start_execution_time, "sec")

In [ ]:
'''
dir_list = os.listdir(CSV_PATH)

print(f"Files in {CSV_PATH} directory: {dir_list}")

# Export to an excel file the files in directory in a column
results_dict = {"File": dir_list, "D": OPTIMAL_D, "C": OPTIMAL_C, "CCR": 96.14, "Error": 3.86}
df = pd.DataFrame(results_dict)
df.to_excel('results.xlsx', index = False)
'''